In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder

In [2]:
diamond=pd.read_csv('diamonds_train.csv')
diamond.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,1.21,Premium,J,VS2,62.4,58.0,4268,6.83,6.79,4.25
1,0.32,Very Good,H,VS2,63.0,57.0,505,4.35,4.38,2.75
2,0.71,Fair,G,VS1,65.5,55.0,2686,5.62,5.53,3.65
3,0.41,Good,D,SI1,63.8,56.0,738,4.68,4.72,3.00
4,1.02,Ideal,G,SI1,60.5,59.0,4882,6.55,6.51,3.95


In [3]:
X=diamond.drop(columns=['price'])
y=diamond.price

In [4]:
X_label=X.apply(LabelEncoder().fit_transform)
X_label.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,101,3,6,5,86,59,305,304,204
1,12,4,4,5,92,49,57,63,54
2,51,0,3,4,117,29,184,178,144
3,21,1,0,2,100,39,90,97,79
4,82,2,3,2,67,69,277,276,174


In [5]:
X_train, X_test, y_train, y_test=train_test_split(X_label, y, test_size=0.2)

In [60]:
gbm = lgb.LGBMRegressor(num_leaves=100,
                        learning_rate=0.05,
                        n_estimators=200)
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l2',
        early_stopping_rounds=50, verbose=False)


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.05, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=200, n_jobs=-1, num_leaves=100, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [61]:
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
print('RMSE:', mean_squared_error(y_test, y_pred) ** 0.5)

RMSE: 523.0881232619217
